### Import packages and Gaia tables

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
from pyvo.dal import TAPService

from astroquery.gaia import Gaia
from astroquery.vizier import Vizier

gaia = TAPService("https://gea.esac.esa.int/tap-server/tap")

#tables = Gaia.load_tables(only_names=True)
#for table in tables:
#    print(table.name)

### Load Gaia_source data

In [2]:
meta_source = Gaia.load_table('gaiadr3.gaia_source')
meta_source
print(meta_source)

meta_galaxies = Gaia.load_table('gaiadr3.galaxy_candidates')
print(meta_galaxies)

#meta_astrophys = Gaia.load.table('gaiadr3.astrophysical_parameters_supp')
#meta_astrophys
#print(meta_astrophys)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
TAP Table name: gaiadr3.galaxy_candidates
Description: This table contains parameters derived from various modules dedicated to the classification and characterisation of sources considered as galaxy candidates. This table has been constructed with the intention to be complete rather than pure and, as such,

# Search Gaia DR3 host galaxy

In [3]:
# Load the id and position information for the recent sources.

sources_df = pd.read_csv(
    "data/sources.csv",
    usecols=["id", "ra [deg]", "dec [deg]", "redshift"]
).rename(columns={
    "ra [deg]": "ra",
    "dec [deg]": "dec",
    "redshift": "z"
})
sources_df

,id,ra,dec,z
0,ZTF25aceekci,179.440246,55.507917,NaN
1,ZTF25aceemtn,183.287855,62.905781,NaN
2,ZTF25aceekrw,180.737327,48.352361,NaN
3,ZTF25aceelbj,169.945969,40.897420,NaN
4,ZTF25acdeiid,37.873606,7.791892,NaN
5,ZTF25acdhetm,142.885962,59.376197,NaN
6,ZTF25acdabfa,11.861833,-6.805553,0.056182
7,ZTF25acczwqv,356.601029,19.681700,NaN
8,ZTF25acceztk,19.383869,16.246087,NaN
9,ZTF25accgmpa,153.461746,39.986779,NaN


# Gaia galaxy candidate table

In [4]:
def host_radius_arcsec(
    z,
    max_kpc=30.0,
    default_arcsec=60.0,
    min_arcsec=10.0,
    max_arcsec=120.0
):
    """Return a reasonable search radius in arcsec given redshift z."""
    if not np.isfinite(z) or z <= 0:
        return default_arcsec

    # Angular diameter distance (Mpc → kpc)
    d_a_kpc = cosmo.angular_diameter_distance(z).to(u.kpc).value

    theta_arcsec = max_kpc / d_a_kpc * 206265.0
    theta_arcsec = max(theta_arcsec, min_arcsec)
    theta_arcsec = min(theta_arcsec, max_arcsec)
    return theta_arcsec


In [5]:
def query_gaia_hosts(ra0, dec0, rdeg, pmin=0.5, top=200):
    """Run a Gaia DR3 galaxy-candidates host-search."""
    adql = f"""
    SELECT TOP {top}
      gc.source_id,
      s.ra, s.dec,
      s.phot_g_mean_mag, s.phot_bp_mean_mag, s.phot_rp_mean_mag, s.bp_rp,
      gc.classlabel_dsc,
      gc.classprob_dsc_combmod_galaxy AS p_gal,
      DISTANCE(POINT('ICRS', s.ra, s.dec),
               POINT('ICRS', {ra0}, {dec0})) * 3600.0 AS sep_arcsec
    FROM gaiadr3.galaxy_candidates AS gc
    JOIN gaiadr3.gaia_source AS s
      ON s.source_id = gc.source_id
    WHERE 1 = CONTAINS(
      POINT('ICRS', s.ra, s.dec),
      CIRCLE('ICRS', {ra0}, {dec0}, {rdeg})
    )
      AND gc.classprob_dsc_combmod_galaxy > {pmin}
      AND gc.classlabel_dsc = 'galaxy'
    ORDER BY sep_arcsec ASC
    """
    return gaia.run_sync(adql).to_table()


In [6]:
all_results = []  # list of per-source DataFrames

for idx, row in sources_df.iterrows():

    src_id = row["id"]
    ra0 = float(row["ra"])
    dec0 = float(row["dec"])
    z = row["z"]

    # dynamic radius
    r_arcsec = host_radius_arcsec(z)
    rdeg = r_arcsec / 3600.0

    print(f"\n=============== {src_id} ===============")
    print(f"RA={ra0:.6f}, Dec={dec0:.6f}, z={z}, r_arcsec={r_arcsec:.1f}\n")

    # 1. Strict query
    galaxy_table = query_gaia_hosts(ra0, dec0, rdeg, pmin=0.5)

    print("Rows returned (p>0.5):", len(galaxy_table))

    # 2. Fallback if needed
    if len(galaxy_table) == 0:
        print("  → Relaxing to p>0.1 and radius×2")
        galaxy_table = query_gaia_hosts(ra0, dec0, 2*rdeg, pmin=0.1)
        print("Rows returned (fallback):", len(galaxy_table))

    if len(galaxy_table) == 0:
        print("  → Still no Gaia galaxy candidates.")
        continue

    df = galaxy_table.to_pandas()
    df["searched_source"] = src_id  # keep track of which transient this is

    display(df)          # still see per-source output
    all_results.append(df)  # <-- collect it

if all_results:
    all_hosts_df = pd.concat(all_results, ignore_index=True)
    all_hosts_df.to_csv("gaia_hosts_all_sources.csv", index=False)
else:
    all_hosts_df = pd.DataFrame()



=============== ZTF25aceekci ===============
RA=179.440246, Dec=55.507917, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 1


,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,classlabel_dsc,p_gal,sep_arcsec,searched_source
0,841033490812198272,179.406787,55.507252,20.321817,18.617582,17.213497,1.404085,galaxy,1.0,68.253685,ZTF25aceekci



=============== ZTF25aceemtn ===============
RA=183.287855, Dec=62.905781, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 1


,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,classlabel_dsc,p_gal,sep_arcsec,searched_source
0,1584253460848878592,183.338158,62.918998,21.106348,19.187916,17.58046,1.607456,galaxy,1.0,95.203291,ZTF25aceemtn



=============== ZTF25aceekrw ===============
RA=180.737327, Dec=48.352361, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 0
  → Still no Gaia galaxy candidates.

=============== ZTF25aceelbj ===============
RA=169.945969, Dec=40.897420, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 0
  → Still no Gaia galaxy candidates.

=============== ZTF25acdeiid ===============
RA=37.873606, Dec=7.791892, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 1


...


=============== ZTF25abzooqy ===============
RA=256.311364, Dec=56.215385, z=0.02918, r_arcsec=49.6

Rows returned (p>0.5): 1


,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,classlabel_dsc,p_gal,sep_arcsec,searched_source
0,1432589438254312576,256.308114,56.219575,19.618868,17.420771,15.902936,1.517835,galaxy,1.0,16.426423,ZTF25abzooqy



=============== ZTF25abztjzq ===============
RA=73.219695, Dec=-1.439102, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 1


,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,classlabel_dsc,p_gal,sep_arcsec,searched_source
0,3226398850302385792,73.194318,-1.417723,20.494902,19.052074,17.580332,1.471743,galaxy,0.983673,119.435771,ZTF25abztjzq



=============== ZTF25abxqirh ===============
RA=85.482906, Dec=-23.142624, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 0
  → Still no Gaia galaxy candidates.

=============== ZTF25abybulv ===============
RA=44.013458, Dec=5.530096, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 0
  → Still no Gaia galaxy candidates.

=============== ZTF25abxrifp ===============
RA=159.269603, Dec=12.214325, z=nan, r_arcsec=60.0

Rows returned (p>0.5): 0
  → Relaxing to p>0.1 and radius×2
Rows returned (fallback): 0
  → Still no Gaia galaxy candidates.


In [7]:
all_hosts_df

,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,classlabel_dsc,p_gal,sep_arcsec,searched_source
0,841033490812198272,179.406787,55.507252,20.321817,18.617582,17.213497,1.404085,galaxy,1.000000,68.253685,ZTF25aceekci
1,1584253460848878592,183.338158,62.918998,21.106348,19.187916,17.580460,1.607456,galaxy,1.000000,95.203291,ZTF25aceemtn
2,19645528305047808,37.871887,7.790526,21.005253,19.035913,17.650644,1.385269,galaxy,0.920592,7.860012,ZTF25acdeiid
3,2522996628714058752,11.854056,-6.794165,21.064892,18.427841,17.029444,1.398397,galaxy,1.000000,49.534045,ZTF25acdabfa
4,2822887287024114688,356.628370,19.697435,20.982460,19.609955,17.956392,1.653563,galaxy,1.000000,108.616038,ZTF25acczwqv
5,2591758638511351040,19.359956,16.245717,21.530910,19.845823,18.322653,1.523170,galaxy,1.000000,82.661827,ZTF25acceztk
6,912677565398856832,133.646702,41.637727,20.211084,18.493324,16.973730,1.519594,galaxy,1.000000,2.436397,ZTF25acbnchv
7,854389395994796800,160.884810,57.672161,19.433056,16.875061,15.358612,1.516449,galaxy,1.000000,50.302052,ZTF25acbepkk
8,381264319895421184,10.460547,41.273932,21.319248,19.243715,17.899590,1.344126,galaxy,0.983673,33.077218,ZTF25acajztg
9,3235015413694244608,76.334762,2.091231,20.045235,17.438900,15.878913,1.559987,galaxy,1.000000,2.357723,ZTF25acaefyx
